In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms, datasets
import torch.nn.functional as F

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize images
])

train_data = datasets.ImageFolder('cats_and_dogs_filtered/train', transform=transform)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)

class CatDogClassifier(nn.Module):
    def __init__(self):
        super(CatDogClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.conv3 = nn.Conv2d(64, 128, 3, 1)
        self.fc1 = nn.Linear(128*6*6, 512)
        self.fc2 = nn.Linear(512, 2)
        
    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2, 2)
        x = torch.relu(self.conv3(x))
        x = torch.max_pool2d(x, 2, 2)
        x = x.view(-1, 128*6*6)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
model = CatDogClassifier()

optimizer = optim.Adam(model.parameters(), lr = 0.001, weight_decay = 0.001)

criterion = nn.CrossEntropyLoss()

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()
        
    epoch_loss = running_loss / len(train_loader)
    accuracy = correct_predictions / total_predictions
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}")

# Observing the impact of regularization on weight parameters
for name, param in model.named_parameters():
    if 'weight' in name:
        print(f'Layer: {name}, L2 Regularization Term: {torch.norm(param, p=2)}')


Epoch 1/10, Loss: 0.6910, Accuracy: 0.5235
Epoch 2/10, Loss: 0.6647, Accuracy: 0.6085
Epoch 3/10, Loss: 0.6418, Accuracy: 0.6400
Epoch 4/10, Loss: 0.6050, Accuracy: 0.6785
Epoch 5/10, Loss: 0.5614, Accuracy: 0.7240
Epoch 6/10, Loss: 0.5357, Accuracy: 0.7285
Epoch 7/10, Loss: 0.5335, Accuracy: 0.7310
Epoch 8/10, Loss: 0.4979, Accuracy: 0.7725
Epoch 9/10, Loss: 0.4715, Accuracy: 0.7805
Epoch 10/10, Loss: 0.4740, Accuracy: 0.7740
Layer: conv1.weight, L2 Regularization Term: 3.214808225631714
Layer: conv2.weight, L2 Regularization Term: 3.5532853603363037
Layer: conv3.weight, L2 Regularization Term: 3.210446834564209
Layer: fc1.weight, L2 Regularization Term: 6.462679386138916
Layer: fc2.weight, L2 Regularization Term: 0.8525404930114746


# ***Q1(b)***

In [6]:
num_epochs = 10
l2_lambda = 0.001  # L2 regularization lambda
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        
        # Compute L2 regularization term
        l2_reg = 0
        for param in model.parameters():
            l2_reg += torch.norm(param, p=2) ** 2
        
        loss = criterion(outputs, labels) + 0.5 * l2_lambda * l2_reg
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()
        
    epoch_loss = running_loss / len(train_loader)
    accuracy = correct_predictions / total_predictions
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}")

# Observing the impact of regularization on weight parameters
for name, param in model.named_parameters():
    if 'weight' in name:
        print(f'Layer: {name}, L2 Regularization Term: {torch.norm(param, p=2)}')

Epoch 1/10, Loss: 0.3905, Accuracy: 0.8400
Epoch 2/10, Loss: 0.3921, Accuracy: 0.8425
Epoch 3/10, Loss: 0.3343, Accuracy: 0.8835
Epoch 4/10, Loss: 0.3187, Accuracy: 0.8860
Epoch 5/10, Loss: 0.2818, Accuracy: 0.9100
Epoch 6/10, Loss: 0.2660, Accuracy: 0.9125
Epoch 7/10, Loss: 0.2107, Accuracy: 0.9440
Epoch 8/10, Loss: 0.1821, Accuracy: 0.9560
Epoch 9/10, Loss: 0.1634, Accuracy: 0.9670
Epoch 10/10, Loss: 0.1575, Accuracy: 0.9645
Layer: conv1.weight, L2 Regularization Term: 3.3872299194335938
Layer: conv2.weight, L2 Regularization Term: 4.1374335289001465
Layer: conv3.weight, L2 Regularization Term: 4.829056262969971
Layer: fc1.weight, L2 Regularization Term: 7.821390628814697
Layer: fc2.weight, L2 Regularization Term: 1.544577956199646


# ***Q2(a)***

In [7]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()
        
    epoch_loss = running_loss / len(train_loader)
    accuracy = correct_predictions / total_predictions
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}")

# Observing the impact of regularization on weight parameters
for name, param in model.named_parameters():
    if 'weight' in name:
        print(f'Layer: {name}, L1 Regularization Term: {torch.norm(param, p=1)}')

Epoch 1/10, Loss: 0.0647, Accuracy: 0.9850
Epoch 2/10, Loss: 0.0553, Accuracy: 0.9880
Epoch 3/10, Loss: 0.0621, Accuracy: 0.9815
Epoch 4/10, Loss: 0.0387, Accuracy: 0.9900
Epoch 5/10, Loss: 0.0297, Accuracy: 0.9940
Epoch 6/10, Loss: 0.0258, Accuracy: 0.9960
Epoch 7/10, Loss: 0.0113, Accuracy: 0.9985
Epoch 8/10, Loss: 0.0073, Accuracy: 1.0000
Epoch 9/10, Loss: 0.0053, Accuracy: 1.0000
Epoch 10/10, Loss: 0.0046, Accuracy: 1.0000
Layer: conv1.weight, L1 Regularization Term: 87.25929260253906
Layer: conv2.weight, L1 Regularization Term: 341.76068115234375
Layer: conv3.weight, L1 Regularization Term: 513.479248046875
Layer: fc1.weight, L1 Regularization Term: 2742.506103515625
Layer: fc2.weight, L1 Regularization Term: 32.98918533325195


# ***Q2(b)***

In [8]:
num_epochs = 10
l1_lambda = 0.001  # L1 regularization lambda
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        
        # Compute L1 regularization term
        l1_reg = 0
        for param in model.parameters():
            l1_reg += torch.norm(param, p=1)
        
        loss = criterion(outputs, labels) + 0.5 * l1_lambda * l1_reg
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()
        
    epoch_loss = running_loss / len(train_loader)
    accuracy = correct_predictions / total_predictions
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}")

# Observing the impact of regularization on weight parameters
for name, param in model.named_parameters():
    if 'weight' in name:
        print(f'Layer: {name}, L1 Regularization Term: {torch.norm(param, p=1)}')

Epoch 1/10, Loss: 2.5349, Accuracy: 0.9775
Epoch 2/10, Loss: 1.8635, Accuracy: 0.8870
Epoch 3/10, Loss: 1.4194, Accuracy: 0.9270
Epoch 4/10, Loss: 1.2141, Accuracy: 0.9355
Epoch 5/10, Loss: 1.1503, Accuracy: 0.9300
Epoch 6/10, Loss: 1.0920, Accuracy: 0.9255
Epoch 7/10, Loss: 1.0224, Accuracy: 0.9495
Epoch 8/10, Loss: 0.9645, Accuracy: 0.9490
Epoch 9/10, Loss: 0.9506, Accuracy: 0.9295
Epoch 10/10, Loss: 0.9203, Accuracy: 0.9475
Layer: conv1.weight, L1 Regularization Term: 87.84284973144531
Layer: conv2.weight, L1 Regularization Term: 222.5211944580078
Layer: conv3.weight, L1 Regularization Term: 283.85113525390625
Layer: fc1.weight, L1 Regularization Term: 813.4820556640625
Layer: fc2.weight, L1 Regularization Term: 31.144742965698242


# ***Q3***

In [9]:
def train_model(model, train_loader, optimizer, criterion, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        accuracy = correct_predictions / total_predictions

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}")

# Without dropout regularization
model_no_dropout = CatDogClassifier()
optimizer_no_dropout = optim.Adam(model_no_dropout.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
print("Training without dropout:")
train_model(model_no_dropout, train_loader, optimizer_no_dropout, criterion)

# With dropout regularization
model_with_dropout = CatDogClassifier()
optimizer_with_dropout = optim.Adam(model_with_dropout.parameters(), lr=0.001)
print("\nTraining with dropout:")
train_model(model_with_dropout, train_loader, optimizer_with_dropout, criterion)

Training without dropout:
Epoch 1/10, Loss: 0.6881, Accuracy: 0.5400
Epoch 2/10, Loss: 0.6597, Accuracy: 0.6020
Epoch 3/10, Loss: 0.6075, Accuracy: 0.6835
Epoch 4/10, Loss: 0.5581, Accuracy: 0.7195
Epoch 5/10, Loss: 0.5083, Accuracy: 0.7525
Epoch 6/10, Loss: 0.4631, Accuracy: 0.7860
Epoch 7/10, Loss: 0.4171, Accuracy: 0.8135
Epoch 8/10, Loss: 0.3607, Accuracy: 0.8400
Epoch 9/10, Loss: 0.2982, Accuracy: 0.8670
Epoch 10/10, Loss: 0.2484, Accuracy: 0.9055

Training with dropout:
Epoch 1/10, Loss: 0.6975, Accuracy: 0.5305
Epoch 2/10, Loss: 0.6774, Accuracy: 0.5595
Epoch 3/10, Loss: 0.6440, Accuracy: 0.6320
Epoch 4/10, Loss: 0.5978, Accuracy: 0.6940
Epoch 5/10, Loss: 0.5786, Accuracy: 0.7015
Epoch 6/10, Loss: 0.5349, Accuracy: 0.7375
Epoch 7/10, Loss: 0.5204, Accuracy: 0.7405
Epoch 8/10, Loss: 0.4729, Accuracy: 0.7670
Epoch 9/10, Loss: 0.4500, Accuracy: 0.7930
Epoch 10/10, Loss: 0.4317, Accuracy: 0.7880


# ***Q4***

In [12]:
class CustomDropout(nn.Module):
    def __init__(self, p=0.5):
        super(CustomDropout, self).__init__()
        self.p = p

    def forward(self, x):
        if not self.training:
            return x
        mask = torch.empty_like(x).bernoulli_(1 - self.p)
        return x * mask / (1 - self.p)
    
class CustomCatDogClassifier(nn.Module):
    def __init__(self):
        super(CustomCatDogClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.conv3 = nn.Conv2d(64, 128, 3, 1)
        self.fc1 = nn.Linear(128*6*6, 512)
        self.fc2 = nn.Linear(512, 2)  # 2 classes: cat and dog
        self.dropout = CustomDropout(0.5)  # Custom dropout with 50% probability

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2, 2)
        x = torch.relu(self.conv3(x))
        x = torch.max_pool2d(x, 2, 2)
        x = x.view(-1, 128*6*6)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)  # Apply custom dropout before the output layer
        x = self.fc2(x)
        return x
    
model_no_custom_dropout = CustomCatDogClassifier()
optimizer_no_custom_dropout = optim.Adam(model_no_custom_dropout.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
print("Training without custom dropout:")
train_model(model_no_custom_dropout, train_loader, optimizer_no_custom_dropout, criterion)

# With custom dropout regularization
model_with_custom_dropout = CustomCatDogClassifier()
optimizer_with_custom_dropout = optim.Adam(model_with_custom_dropout.parameters(), lr=0.001)
print("\nTraining with custom dropout:")
train_model(model_with_custom_dropout, train_loader, optimizer_with_custom_dropout, criterion)


Training without custom dropout:
Epoch 1/10, Loss: 0.6860, Accuracy: 0.5430
Epoch 2/10, Loss: 0.6541, Accuracy: 0.6095
Epoch 3/10, Loss: 0.6106, Accuracy: 0.6680
Epoch 4/10, Loss: 0.5602, Accuracy: 0.7145
Epoch 5/10, Loss: 0.4979, Accuracy: 0.7625
Epoch 6/10, Loss: 0.4589, Accuracy: 0.7940
Epoch 7/10, Loss: 0.4038, Accuracy: 0.8160
Epoch 8/10, Loss: 0.3490, Accuracy: 0.8520
Epoch 9/10, Loss: 0.3400, Accuracy: 0.8465
Epoch 10/10, Loss: 0.2623, Accuracy: 0.8845

Training with custom dropout:
Epoch 1/10, Loss: 0.6895, Accuracy: 0.5420
Epoch 2/10, Loss: 0.6653, Accuracy: 0.5985
Epoch 3/10, Loss: 0.6252, Accuracy: 0.6590
Epoch 4/10, Loss: 0.5653, Accuracy: 0.6985
Epoch 5/10, Loss: 0.5319, Accuracy: 0.7270
Epoch 6/10, Loss: 0.4867, Accuracy: 0.7650
Epoch 7/10, Loss: 0.4366, Accuracy: 0.8065
Epoch 8/10, Loss: 0.4000, Accuracy: 0.8215
Epoch 9/10, Loss: 0.3343, Accuracy: 0.8540
Epoch 10/10, Loss: 0.2816, Accuracy: 0.8800


# ***Q5***

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import torchvision
from torchvision import transforms, datasets

# Define transforms
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize images
])

# Load data
train_data = datasets.ImageFolder('path_to_train_data', transform=transform)
train_size = int(0.8 * len(train_data))
val_size = len(train_data) - train_size
train_data, val_data = random_split(train_data, [train_size, val_size])

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64)

# Define the neural network architecture
class CatDogClassifier(nn.Module):
    def __init__(self):
        super(CatDogClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.conv3 = nn.Conv2d(64, 128, 3, 1)
        self.fc1 = nn.Linear(128*6*6, 512)
        self.fc2 = nn.Linear(512, 2)  # 2 classes: cat and dog

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2, 2)
        x = torch.relu(self.conv3(x))
        x = torch.max_pool2d(x, 2, 2)
        x = x.view(-1, 128*6*6)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Training function with early stopping
def train_with_early_stopping(model, train_loader, val_loader, criterion, optimizer, patience=5, num_epochs=50):
    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Validate
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        val_loss /= len(val_loader)

        print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {running_loss / len(train_loader):.4f}, "
              f"Validation Loss: {val_loss:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"Validation loss hasn't improved for {patience} epochs. Early stopping...")
            break

# Without early stopping
model_no_early_stopping = CatDogClassifier()
optimizer_no_early_stopping = optim.Adam(model_no_early_stopping.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
print("Training without early stopping:")
train_with_early_stopping(model_no_early_stopping, train_loader, val_loader, criterion, optimizer_no_early_stopping)

# With early stopping
model_with_early_stopping = CatDogClassifier()
optimizer_with_early_stopping = optim.Adam(model_with_early_stopping.parameters(), lr=0.001)
print("\nTraining with early stopping:")
train_with_early_stopping(model_with_early_stopping, train_loader, val_loader, criterion, optimizer_with_early_stopping)
